In [19]:
import pdfplumber
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
import re
import string
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [24]:
def vectorizeText(inputText):
    
    # get counts and remove stop words
    cv = CountVectorizer(stop_words='english')
    X = cv.fit_transform(inputText)
    df_cv = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())  
    df_cv.info()
    print(df_cv.columns)
    print(df_cv)

    # get term frequencies and remove stop words
#     cv_tfidf = TfidfVectorizer(stop_words='english')
#     X_tfidf = cv_tfidf.fit_transform(page_sentences)
#     df_tfidf = pd.DataFrame(X_tfidf.toarray(), columns=cv_tfidf.get_feature_names())
#     df_tfidf.info()
#     print(df_tfidf.columns)
#     print(df_tfidf)      


def cleanText(page):
    page_text = this_page.extract_text()
    
    clean_text = page_text
    
    # remove punctuation
    clean_text = re.sub('[%s]' % re.escape(string.punctuation), ' ', clean_text)
    # make lower case
    clean_text = clean_text.lower()
    
    # remove numbers
    clean_text = re.sub('\w*\d\w*', ' ', clean_text)
      
    # tokenize the text either in individual words or sentences
    clean_text = word_tokenize(clean_text)
#     page_sentences = sent_tokenize(clean_text) # this doesn't work if punctuation has already been removed, can't clean the punctuation with the same line of code as above once it's been tokenized

    # remove punctuation which has been reintroduced after tokenizing
    clean_text = [word for word in clean_text if word.isalpha()]

    # remove stop words (can do this manually or automatically with the vectorizer)
    clean_text = [word for word in clean_text if word not in stop_words]
    
    # stem words
    clean_text = [porter.stem(word) for word in clean_text]

    
    print(clean_text)
#     print(page_sentences)
    
#     print('Num words on page = ', len(page_words))
#     print('Num sentences on page = ', len(page_sentences))
    
#     twograms = list(ngrams(page_words,2))
#     print(twograms)

    return clean_text
    

    

In [27]:
with pdfplumber.open("../StormlightArchiveBooks/The_Way_of_Kings_The_Stormlight_Archive_Book_1.pdf") as pdf:
    this_page = pdf.pages[16]
    cleaned_page_text = cleanText(this_page)
    vectorizeText(cleaned_page_text)
    
    


['map', 'alethkar', 'surround', 'creat', 'majesti', 'gavilar', 'kholin', 'royal', 'surveyor', 'circa']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   alethkar  10 non-null     int64
 1   circa     10 non-null     int64
 2   creat     10 non-null     int64
 3   gavilar   10 non-null     int64
 4   kholin    10 non-null     int64
 5   majesti   10 non-null     int64
 6   map       10 non-null     int64
 7   royal     10 non-null     int64
 8   surround  10 non-null     int64
 9   surveyor  10 non-null     int64
dtypes: int64(10)
memory usage: 928.0 bytes
Index(['alethkar', 'circa', 'creat', 'gavilar', 'kholin', 'majesti', 'map',
       'royal', 'surround', 'surveyor'],
      dtype='object')
   alethkar  circa  creat  gavilar  kholin  majesti  map  royal  surround  \
0         0      0      0        0       0        0    1      0         0   
1         1   